## Important notes:
- looping through the alignment doesn't give reads but alignments and one read may align the reference in more than one position
- this output doesn't include secondary alignments but these may be present in others

In [1]:
from pathlib import Path
import pysam
import argparse

In [2]:
    def get_regions(query_sequence:str, cigar_string:str) -> dict:
        '''move across the read and bin by alignment type (clipped or aligned)''' 
        # could potentially use inverse of pysam.AlignedSegment.query_alignment_sequence
        # this is seq[qstart:qend] and excludes soft-clipped bases, thus using everything outside should give seqs
        
        read_consuming_op_ints = [0, 1, 4, 7, 8]
        read_consuming_ops = ["M", "I", "S", "=", "X"]
        # M is alignment match, = is sequence match,  I is insertion, S i substitution and X is mismatch
        # to hold clipped and aligned regions
        seq_extract = {op: [] for op in read_consuming_ops}
        # current position in read walk
        pos = 0 
        for c in cigar_string:
            # seems to extract e.g. 30S from cigar string
            # can be done using pysam.AlignedSegment.cigartuples but letters are encoded by numbers
            op_int, length = c

            if op_int in read_consuming_op_ints:

                # get the op as a str char 
                op = read_consuming_ops[read_consuming_op_ints.index(op_int)]
                if query_sequence:
                    seq_region = query_sequence[pos:pos+length]
                    seq_extract[op].append(seq_region)
                pos += length

        return seq_extract

In [3]:
sam_path = "/Users/vincentn/Downloads/barcode02_1kb_to_pWEB_TNC.sam"

### Version that includes unmapped reads

In [4]:
# This version is derived from a script written by Matt Storey
# with open(output_path, 'a') as fa:
SAM_FILE = pysam.AlignmentFile(sam_path, "r") # open samfile        
#count the reads for debug
read_number = 0
ID_list = []
counter = 0
# Note that this loops through alignments and not reads!
for read in SAM_FILE:
    # get the mapping data for each read 
    cigar_string = read.cigar
    query_sequence = read.query_sequence
    query_name = read.query_name
    # get unmapped reads
    if read.is_unmapped == True:
        #print(query_name)
        #print(query_sequence)
        read_number += 1
    # This check seems to be necessary 
    elif query_sequence and cigar_string:
        seq_extract_dict = get_regions(query_sequence, cigar_string)
        #print(read_number)
        #print(query_name)
        #print(cigar_string)
        #print(seq_extract_dict)
        #print("")
        #read_number += 1
            # NOTE cigar string contains quite a few hard-clipped sequences -> are omitted!?
            # Would probably need to change the behavior of the aligner
            # --hard-mask-level and -M
            # -Y	In SAM output, use soft clipping for supplementary alignments.
            
#             # grab the softclipped bits and write them to the out file 
        for i, soft_clip in enumerate(seq_extract_dict["S"]):
            ID_string = ">"+query_name+"_"+str(i)
            if ID_string not in ID_list:
                read_number += 1
                #print(">"+query_name+"_"+str(i))
                ID_list.append(ID_string)
            else:
                counter += 1
                #print(">"+query_name+"_"+str(counter))
                ID_list.append(">"+query_name+"_"+str(counter))
#                 fa.write(f">{query_name}_{i}")
#                 fa.write("\n")
#                 fa.write(soft_clip)
#                 fa.write('\n')
print(read_number)
print(counter)


17288
22725


In [5]:
print(len(ID_list))
print(len(set(ID_list)))

33580
33579


In [6]:
number_of_reads = 0
number_of_unmapped_reads = 0
number_of_mapped_reads =0
SAM_FILE = pysam.AlignmentFile(sam_path, "r") # open samfile
for read in SAM_FILE:
    number_of_reads += 1
    if read.is_unmapped == True:
        number_of_unmapped_reads += 1
        #print(read.query_name)
        #print(read.cigar)
    elif read.is_unmapped == False:        
        number_of_mapped_reads += 1
print(number_of_reads)
print(number_of_unmapped_reads)
print(number_of_mapped_reads)

24509
6433
18076


### Logic-based version (alt2)

The idea is to have a script that checks mapped reads for the only possible conformations of vector and insert to discard any technical artefacts. Possible conformations are:
- Insert only
- Vector only
- Vector at start only
- Vector at end only
- Whole vector sequence in the middle
- Vector at start and end

In [4]:
# NOTE THAT THIS CREATES EMPTY READS AND REQUIRES POST-FILTERING
# read qualities (as previously) but also CIGAR strings are disregarded in this version 
# with open(output_path, 'a') as fa:
SAM_FILE = pysam.AlignmentFile(sam_path, "r") # open samfile        
algn_dict = {}
# Note that this loops through alignments and not reads!
for read in SAM_FILE:
    #print(read.query_name,read.query_length,read.query_alignment_start,read.query_alignment_end,read.query_alignment_length)
    # read.infer_query_length() includes reads that are unmapped as None for query length -> seems to be only supplementary alignments
    if read.query_name in algn_dict:
        algn_dict[read.query_name].append((read.query_alignment_start,read.query_alignment_end))
    else:
        algn_dict[read.query_name] = [read.infer_query_length(),(read.query_alignment_start,read.query_alignment_end)]

SAM_FILE = pysam.AlignmentFile(sam_path, "r") 
discarded_count = 0
more_than_3_count = 0
for read in SAM_FILE:
    seq = read.query_sequence
    name = read.query_name
    query_length = algn_dict[read.query_name][0]
    # Unmapped reads
    if query_length == None:
#         fa.write(f">{name}_{1}")
#         fa.write("\n")
#         fa.write(seq)
#         fa.write('\n')
        print("unmapped")
    # Single alignment
    elif len(algn_dict[name]) == 2:
        start = algn_dict[name][1][0]
        end = algn_dict[name][1][1]
        if len(seq) == end-start:
            discarded_count += 1
        elif start < 500 and len(seq[end:query_length]) > 1000:
#                 fa.write(f">{name}_{1}")
#                 fa.write("\n")
#                 fa.write(seq[algn_dict[name][1][1]:algn_dict[name][0]])
#                 fa.write('\n')
            print("left")
            print(len(seq[end:query_length]))
        elif query_length-end < 500 and len(seq[0:start]) > 1000:
#                 fa.write(f">{name}_{1}")
#                 fa.write("\n")
#                 fa.write(seq[0:algn_dict[name][1][0]])
#                 fa.write('\n')
            print("right")
            print(len(seq[0:start]))
        # <250 bp alignment in middle of >1kb reads considered a misalignment here
        elif end-start < 250 and len(seq) > 1000:
#                 fa.write(f">{name}_{1}")
#                 fa.write("\n")
#                 fa.write(seq)
#                 fa.write('\n')
            print("misalign")
            print(len(seq))
        # Using the 3500 from below doesn't change number much
        elif end-start < 6500 and len(seq[0:start]) > 1000 and len(seq[end:query_length]) > 1000:                
#                 fa.write(f">{name}_{1}")
#                 fa.write("\n")
#                 fa.write(seq[0:algn_dict[name][1][0]])
#                 fa.write('\n')
#                 fa.write(f">{name}_{2}")
#                 fa.write("\n")
#                 fa.write(seq[algn_dict[name][1][1]:algn_dict[name][0]])
#                 fa.write('\n')
            print("middle")
            print(len(seq[0:algn_dict[name][1][0]]),len(seq[algn_dict[name][1][1]:algn_dict[name][0]]))
    # Two alignments and >20 kb (mostly insert; changing to >10kb doesn't change numbers much)
    elif len(algn_dict[name]) == 3 and query_length > 20000:
        # determine max and min values in alignment
        max_left = max(algn_dict[name][1:],key=lambda item:item[1])[0]
        min_left = min(algn_dict[name][1:],key=lambda item:item[1])[0]
        min_right = min(algn_dict[name][1:],key=lambda item:item[1])[1]
        max_right = max(algn_dict[name][1:],key=lambda item:item[1])[1]
        # case where vector is at both ends and full insert is sequenced (98 of these)
        if max_left-min_right > 20000:
#                 fa.write(f">{name}_{1}")
#                 fa.write("\n")
#                 fa.write(seq[min_right:max_left])
#                 fa.write('\n')
            print(algn_dict[name])
            print(len(seq[min_right:max_left]))
        # case where vector is in middle
        if max_right-min_left < 6500:
#                 fa.write(f">{name}_{1}")
#                 fa.write("\n")
#                 fa.write(seq[0:min_left])
#                 fa.write('\n')
#                 fa.write(f">{name}_{2}")
#                 fa.write("\n")
#                 fa.write(seq[max_right:query_length])
#                 fa.write('\n')
            print(algn_dict[name])            
            print(len(seq[0:min_left]),len(seq[max_right:query_length]))
    elif len(algn_dict[name]) > 3:
        more_than_3_count += 1
    else:
        discarded_count += 1

print(more_than_3_count)
print(discarded_count)
print(len(algn_dict))

unmapped
unmapped
unmapped
right
1013
unmapped
right
16180
unmapped
left
4631
unmapped
right
13296
unmapped
unmapped
unmapped
unmapped
misalign
1986
right
4282
unmapped
unmapped
unmapped
unmapped
unmapped
right
14580
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
middle
4246 1897
unmapped
unmapped
unmapped
unmapped
left
17950
unmapped
unmapped
unmapped
left
3001
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
right
1483
unmapped
unmapped
unmapped
unmapped
unmapped
left
4438
unmapped
left
2185
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
right
27324
left
4584
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
middle
4444 2493
unmapped
unmapped
right
2207
unmapped
unmapped
unmapped
unmapped
unmapped
left
2944
unmapped
unmapped
unmapped
unmapp

left
6014
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
middle
7063 3079
right
1955
unmapped
unmapped
unmapped
[50738, (42434, 45862), (3418, 3801)]
38633
[50738, (42434, 45862), (3418, 3801)]
38633
left
1528
right
4691
unmapped
left
10144
unmapped
middle
3457 13330
right
3987
unmapped
unmapped
left
3346
unmapped
unmapped
unmapped
[43724, (32332, 35738), (2, 434)]
31898
[43724, (32332, 35738), (2, 434)]
31898
unmapped
middle
35762 2219
unmapped
unmapped
left
3230
unmapped
unmapped
unmapped
left
2781
unmapped
middle
4480 2368
left
21369
unmapped
unmapped
unmapped
unmapped
right
9442
unmapped
left
4457
unmapped
unmapped
unmapped
unmapped
unmapped
left
2536
unmapped
unmapped
unmapped
right
10113
left
6162
unmapped
unmapped
unmapped
left
26526
unmapped
unmapped
unmapped
unmapped
unmapped
middle
2717 3434
unmapped
unmapped
unmapped
unmapped
right
13856
unmapped
right
14889
left
18890
unmapped
unmapped
middle
7247 3159
unmapped
unmapped
right
2511
left
1271

unmapped
unmapped
unmapped
unmapped
unmapped
middle
1176 2153
left
2121
unmapped
unmapped
unmapped
unmapped
right
4610
unmapped
unmapped
unmapped
left
19614
unmapped
unmapped
unmapped
left
1446
unmapped
unmapped
unmapped
unmapped
middle
2681 1514
unmapped
left
1751
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
left
19496
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
middle
3717 2667
left
7476
unmapped
unmapped
left
18925
unmapped
unmapped
unmapped
unmapped
unmapped
left
3363
middle
12606 2399
right
14395
unmapped
left
18982
unmapped
unmapped
unmapped
unmapped
unmapped
right
7957
left
4015
unmapped
unmapped
unmapped
right
2709
unmapped
middle
3558 6021
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped


unmapped
unmapped
left
1257
unmapped
left
3530
unmapped
left
11072
right
1542
unmapped
unmapped
unmapped
unmapped
middle
3439 20455
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
middle
4071 1435
right
21911
unmapped
unmapped
unmapped
right
6426
unmapped
unmapped
right
1630
left
5910
left
4434
unmapped
unmapped
left
2046
unmapped
unmapped
unmapped
right
3504
unmapped
unmapped
left
6713
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
[39680, (35866, 39298), (174, 232)]
35634
[39680, (35866, 39298), (174, 232)]
35634
[44321, (4879, 7216), (43045, 44319)]
35829
[44321, (4879, 7216), (43045, 44319)]
35829
unmapped
unmapped
middle
2110 1914
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
left
7634
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
left
2692
middle
5904 2052
left
3835

unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
left
2468
unmapped
unmapped
unmapped
middle
4222 7191
unmapped
unmapped
unmapped
left
12541
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
right
4907
unmapped
left
1079
unmapped
unmapped
unmapped
[32338, (25928, 29357), (29353, 29710)]
25928 2628
[32338, (25928, 29357), (29353, 29710)]
25928 2628
left
2211
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
[37415, (0, 4586), (1007, 1726)]
1007 32829
[37415, (0, 4586), (1007, 1726)]
1007 32829
unmapped
unmapped
unmapped
right
3793
unmapped
left
2198
unmapped
unmapped
unmapped
middle
3936 3791
unmapped
unmapped
left
8029
right
17353
unmapped
unmapped
unmapped
unmapped
right
9209
[34487, (0, 2183), (34258, 34472)]
32075
[34487, (0, 2183), (34258, 34472)]
32075
unmapped
right
6440
middle
4934 11312
right
26777
unmapped
[28973, (2564, 3258), (26406, 26527)]
23148
[28973, (2564, 3258), (26406, 26527)]
23148
unmapped
unmapped


unmapped
left
3223
unmapped
unmapped
unmapped
unmapped
unmapped
left
23409
unmapped
unmapped
unmapped
unmapped
unmapped
right
13596
unmapped
left
18660
middle
23959 1777
unmapped
left
13416
unmapped
unmapped
middle
6065 2734
unmapped
unmapped
right
9886
unmapped
middle
1015 1017
unmapped
unmapped
right
3957
unmapped
unmapped
unmapped
right
11000
middle
10810 1059
right
18027
unmapped
unmapped
left
21237
unmapped
unmapped
unmapped
unmapped
right
6396
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
middle
4360 27015
unmapped
unmapped
unmapped
left
16253
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
right
3044
left
11279
unmapped
unmapped
unmapped
unmapped
left
1991
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped

unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
middle
22804 6011
middle
2338 1043
right
4834
unmapped
unmapped
left
11733
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
left
4300
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
middle
6178 1245
unmapped
unmapped
unmapped
unmapped
unmapped
left
1661
unmapped
unmapped
unmapped
unmapped
left
13521
unmapped
unmapped
unmapped
right
3734
unmapped
unmapped
unmapped
middle
8373 2992
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
left
7624
unmapped
unmapped
right
5749
left
4257
left
3613
unmapped
unmapped
unmapped
unmapped
right
5445
right
3851
[24395, (17620, 21014), (21051, 21415)]
17620 2980
[24395, (17620, 21014), (21051, 21415)]
17620 2980
unmapped
unmapped
unmapped
unmapped
unmapped
middle
10

unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
right
6256
unmapped
unmapped
left
5823
unmapped
unmapped
middle
2298 3309
unmapped
left
3011
unmapped
unmapped
unmapped
right
4086
unmapped
right
4788
unmapped
unmapped
unmapped
left
2808
unmapped
unmapped
unmapped
left
4023
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
left
9436
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
left
1223
unmapped
unmapped
left
1424
unmapped
unmapped
left
3675
unmapped
unmapped
right
9943
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
right
3689
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
middle
8746 1188
unmapped
unmapped
unmapped
right
1260
unmapped
unmapped
left
10284
unmapped
unmapped
left
5726
unmapped
unmapped
unmapped
unmapped
unmapped
right
5760
unmapped
unmapped
middle
23997 6111
unmapped
unmapped
middle
3248 7263
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
right
6896
unma

[30433, (2973, 5321), (27460, 27825)]
22139
[30433, (2973, 5321), (27460, 27825)]
22139
unmapped
unmapped
unmapped
unmapped
middle
2510 6185
unmapped
unmapped
[38160, (5471, 7836), (32689, 33049)]
24853
[38160, (5471, 7836), (32689, 33049)]
24853
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
left
4312
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
middle
4371 2023
unmapped
unmapped
unmapped
unmapped
middle
16666 3697
unmapped
[25211, (16899, 20319), (20315, 20676)]
16899 4535
[25211, (16899, 20319), (20315, 20676)]
16899 4535
unmapped
unmapped
middle
8278 5533
right
4488
unmapped
unmapped
unmapped
left
2421
unmapped
unmapped
unmapped
unmapped
[30462, (23636, 27035), (27043, 27413)]
23636 3049
[30462, (23636, 27035), (27043, 27413)]
23636 3049
right
5055
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
right
9583
unmapped
unmapped
right
9959
unmapped
unmapped
left
1309
unmapped
[32941, (32156, 32941), (5, 501)]
31

unmapped
unmapped
unmapped
unmapped
right
12556
unmapped
right
29661
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
right
3967
unmapped
unmapped
unmapped
unmapped
left
2577
unmapped
unmapped
left
9743
unmapped
unmapped
unmapped
unmapped
unmapped
middle
1486 5966
unmapped
unmapped
unmapped
unmapped
left
3377
unmapped
middle
1697 20404
unmapped
unmapped
unmapped
unmapped
left
6855
unmapped
unmapped
unmapped
unmapped
right
9831
unmapped
unmapped
unmapped
left
10566
right
19354
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
left
14552
middle
6893 4674
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
left
3574
unmapped
unmapped
unmapped
unmapped
right
2721
[28871, (24992, 28428), (118, 443)]
24549
[28871, (24992, 28428), (118, 443)]
24549
unmapped
unmapped
unmapped
unmapped
right
7957
middle
7231 4607
left
13769
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
middle
2039 4396
unmapped
u

### Logic-based version 2 (alt3)

In [27]:
# NOTE THAT THIS CREATES EMPTY READS AND REQUIRES POST-FILTERING
# read qualities (as previously) but also CIGAR strings are disregarded in this version 
# with open(output_path, 'a') as fa:
SAM_FILE = pysam.AlignmentFile(sam_path, "r") # open samfile        
algn_dict = {}
# Note that this loops through alignments and not reads!
for read in SAM_FILE:
    #print(read.query_name,read.query_length,read.query_alignment_start,read.query_alignment_end,read.query_alignment_length)
    # read.infer_query_length() includes reads that are unmapped as None for query length
    if read.query_name in algn_dict:
        algn_dict[read.query_name].append((read.query_alignment_start,read.query_alignment_end))
    else:
        algn_dict[read.query_name] = [read.infer_query_length(),(read.query_alignment_start,read.query_alignment_end)]

SAM_FILE = pysam.AlignmentFile(sam_path, "r") 
discarded_count = 0
written_list = []
for read in SAM_FILE:
    seq = read.query_sequence
    name = read.query_name
    query_length = algn_dict[read.query_name][0]
    # Unmapped reads
    if query_length == None:
#         fa.write(f">{name}_{1}")
#         fa.write("\n")
#         fa.write(seq)
#         fa.write('\n')
        print("unmapped")
    # Single alignment
    elif len(algn_dict[name]) == 2:
        start = algn_dict[name][1][0]
        end = algn_dict[name][1][1]
        if len(seq) == end-start:
            discarded_count += 1
        elif start < 500 and len(seq[end:query_length]) > 1000:
#                 fa.write(f">{name}_{1}")
#                 fa.write("\n")
#                 fa.write(seq[algn_dict[name][1][1]:algn_dict[name][0]])
#                 fa.write('\n')
            print("left")
            print(query_length,len(seq[end:query_length]))
        elif query_length-end < 500 and len(seq[0:start]) > 1000:
#                 fa.write(f">{name}_{1}")
#                 fa.write("\n")
#                 fa.write(seq[0:algn_dict[name][1][0]])
#                 fa.write('\n')
            print("right")
            print(query_length,len(seq[0:start]))
        # Changed from 250 bp in alt2. <500 bp alignment in middle of >1kb reads considered a misalignment here
        elif end-start < 500 and len(seq) > 1000:
#                 fa.write(f">{name}_{1}")
#                 fa.write("\n")
#                 fa.write(seq)
#                 fa.write('\n')
            print("misalign")
            print(len(seq))
        # Using the 3500 from below doesn't change number much
        elif end-start < 6500 and len(seq[0:start]) > 1000 and len(seq[end:query_length]) > 1000:                
#                 fa.write(f">{name}_{1}")
#                 fa.write("\n")
#                 fa.write(seq[0:algn_dict[name][1][0]])
#                 fa.write('\n')
#                 fa.write(f">{name}_{2}")
#                 fa.write("\n")
#                 fa.write(seq[algn_dict[name][1][1]:algn_dict[name][0]])
#                 fa.write('\n')
            print("middle")
            print(query_length,len(seq[0:algn_dict[name][1][0]]),len(seq[algn_dict[name][1][1]:algn_dict[name][0]]))
    # Reads above 20000 bp to be filtered for misalignments where "true" alignments are >500 bp
    elif query_length > 20000:
        # Make a list of reads already written, need to do this if >1 alignment of a read is in SAM file
        if name not in written_list:
            pos_start=[]
            pos_end=[]
            # Make a list of "true" alignment positions
            for pos in algn_dict[name][1:]:
                if pos[1]-pos[0] > 500:
                    pos_start.append(pos[0])
                    pos_end.append(pos[1])
            # Read with no "true" alignments
            if len(pos_start) == 0:
                print("misalign_large")
                print(len(seq))
                written_list.append(name)
    #                 fa.write(f">{name}_{1}")
    #                 fa.write("\n")
    #                 fa.write(seq)
    #                 fa.write('\n')
            if len(pos_start) == 1:
                if pos_start[0] < 500:
                    print("large_left")
                    print(query_length,len(seq[pos_end[0]:algn_dict[name][0]]))
                    written_list.append(name)
    #                 fa.write(f">{name}_{1}")
    #                 fa.write("\n")
    #                 fa.write(seq[pos_end[0]:algn_dict[name][0]])
    #                 fa.write('\n')
                elif algn_dict[i][0]-pos_end[0] < 500:
                    print("large_right")
                    print(query_length,len(seq[0:pos_start[0]]))
                    written_list.append(name)
    #                 fa.write(f">{name}_{1}")
    #                 fa.write("\n")
    #                 fa.write(seq[0:pos_start[0]])
    #                 fa.write('\n')
                elif pos_end[0]-pos_start[0] < 6500:
                    print("large_middle")
                    print(query_length,len(seq[0:pos_start[0]]),len(seq[pos_end[0]:algn_dict[name][0]]))
                    written_list.append(name)
    #                 fa.write(f">{name}_{1}")
    #                 fa.write("\n")
    #                 fa.write(seq[0:pos_start[0]])
    #                 fa.write('\n')
    #                 fa.write(f">{name}_{2}")
    #                 fa.write("\n")
    #                 fa.write(seq[pos_end[0]:algn_dict[name][0]])
    #                 fa.write('\n')
            elif len(pos_start) == 2:
                large_double += 1
                if min(pos_start) < 1000:
                    print("large_left")
                    print(query_length,len(seq[min(pos_end):max(pos_start)]),len(seq[max(pos_end):algn_dict[name][0]]))
                    written_list.append(name)
    #                 fa.write(f">{name}_{1}")
    #                 fa.write("\n")
    #                 fa.write(seq[min(pos_end):max(pos_start)])
    #                 fa.write('\n')
    #                 fa.write(f">{name}_{2}")
    #                 fa.write("\n")
    #                 fa.write(seq[max(pos_end):algn_dict[name][0]])
    #                 fa.write('\n')
                elif algn_dict[i][0]-max(pos_end) < 1000:
                    print("large_right")
                    print(query_length,len(seq[0:min(pos_start)]),len(seq[min(pos_end):max(pos_start)]))
                    written_list.append(name)
    #                 fa.write(f">{name}_{1}")
    #                 fa.write("\n")
    #                 fa.write(seq[0:min(pos_start)])
    #                 fa.write('\n')
    #                 fa.write(f">{name}_{2}")
    #                 fa.write("\n")
    #                 fa.write(seq[min(pos_end):max(pos_start)])
    #                 fa.write('\n')
                else:
                    print("large_middle")
                    print(query_length,len(seq[0:min(pos_start)]),len(seq[min(pos_end):max(pos_start)]),len(seq[max(pos_end):algn_dict[i][0]]))
                    written_list.append(name)
    #                 fa.write(f">{name}_{1}")
    #                 fa.write("\n")
    #                 fa.write(seq[0:min(pos_start)])
    #                 fa.write('\n')
    #                 fa.write(f">{name}_{2}")
    #                 fa.write("\n")
    #                 fa.write(seq[min(pos_end):max(pos_start)])
    #                 fa.write('\n')
    #                 fa.write(f">{name}_{3}")
    #                 fa.write("\n")
    #                 fa.write(seq[max(pos_end):algn_dict[i][0]])
    #                 fa.write('\n') 
    else:
        discarded_count += 1

print(discarded_count)
print(len(algn_dict))

unmapped
unmapped
unmapped
right
4316 1013
unmapped
right
19770 16180
unmapped
left
5503 4631
unmapped
right
16777 13296
large_middle
33974 1287 29159 95
unmapped
unmapped
unmapped
unmapped
misalign
1986
right
4941 4282
unmapped
unmapped
unmapped
large_middle
40559 11923 6635 14579
large_middle
41316 5686 27102 1016
unmapped
unmapped
right
15223 14580
unmapped
unmapped
unmapped
unmapped
large_right
41974 5856 29521
large_middle
41395 26430 7165 2006
unmapped
unmapped
unmapped
unmapped
middle
6491 4246 1897
unmapped
unmapped
unmapped
unmapped
left
18597 17950
large_right
41203 27126 6706
unmapped
unmapped
large_middle
29252 10174 7193 6085
unmapped
left
3156 3001
unmapped
unmapped
unmapped
large_middle
28321 9167 7290 6116
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
right
2356 1483
unmapped
unmapped
unmapped
unmapped
unmapped
large_middle
40009 22284 6714 5241
left
6174 4438
unmapped
left
2537 2185
unmapped
unmapped
unmappe

unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
right
3169 1920
unmapped
unmapped
unmapped
unmapped
large_left
43566 30288 7693
large_middle
42046 17401 6710 11734
unmapped
large_left
25502 6654 15086
large_left
48491 36002 6870
unmapped
unmapped
large_middle
39954 2509 29499 2148
unmapped
right
29227 29044
unmapped
unmapped
unmapped
large_middle
24093 5987 7652 4710
large_middle
30915 6242 7452 11466
unmapped
unmapped
left
1550 1325
unmapped
unmapped
unmapped
unmapped
large_left
47102 6692 35314
unmapped
unmapped
unmapped
left
28859 27623
unmapped
unmapped
unmapped
unmapped
unmapped
middle
13364 1161 9863
misalign
1080
unmapped
right
7365 4473
unmapped
unmapped
unmapped
right
5066 3989
large_middle
48761 23259 6731 5815
unmapped
large_middle
44800 33330 8041
unmapped
unmapped
unmapped
unmapped
unmapped
right
13811 12938
unmapped
unmapped
large_middle
48398 7600 6752 21493
large_right
48187 28419 6689
large_left
48801 36428 6075
large_middle
38231 19547 13451 4
unmapped
large_mid

unmapped
large_middle
41465 22602 6683 6406
large_right
46009 29514 6702
unmapped
large_right
46244 2760 33736
unmapped
unmapped
unmapped
right
11685 8661
left
4468 4162
unmapped
unmapped
large_left
43672 7587 29864
left
3129 1036
unmapped
unmapped
unmapped
unmapped
right
5311 4730
large_right
46029 2596 33630
unmapped
large_middle
24689 10060 6626 2231
unmapped
unmapped
large_middle
26628 15484 6858 169
unmapped
right
13910 13296
unmapped
large_middle
33156 14000 7557 5837
unmapped
unmapped
right
5092 3531
large_middle
42002 5078 6696 24056
unmapped
unmapped
unmapped
unmapped
left
6602 3187
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
left
2673 2348
unmapped
unmapped
middle
11257 4171 3653
unmapped
middle
22946 14012 5538
right
12408 10938
unmapped
right
5205 3579
unmapped
unmapped
unmapped
middle
21768 3259 16153
unmapped
unmapped
large_left
38541 7492 27321
unmapped
unmapped
unmapped
unmapped
unmapped
left
15458 15210
large_middle
48410 17022 6725

unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
large_middle
38031 1064 7198 23993
unmapped
unmapped
right
6401 4502
unmapped
middle
20547 11051 6104
large_middle
42072 16074 6692 13082
large_left
27981 7519 15110
unmapped
unmapped
unmapped
unmapped
unmapped
large_left
40940 28176 7548
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
large_middle
41872 1042 29436 5368
unmapped
unmapped
large_middle
43190 4148 7727 23952
unmapped
unmapped
unmapped
unmapped
unmapped
middle
9538 1482 5753
unmapped
unmapped
unmapped
large_middle
41851 8509 7505 19859
large_middle
40822 25114 6622 1608
unmapped
large_middle
20798 8744 8 8968
unmapped
unmapped
unmapped
unmapped
unmapped
right
8186 6081
right
5524 2267
unmapped
unmapped
large_middle
36196 9016 12073 10754
unmapped
unmapped
unmapped
unmapped
large_right
41149 26993 6723
large_middle
33478 88

unmapped
large_middle
46290 14277 6743 14793
unmapped
unmapped
large_middle
45548 3336 0 32681
unmapped
left
5200 4841
unmapped
large_right
45379 3577 33740
large_middle
25483 6758 6597 6394
middle
23900 3365 18177
unmapped
left
5745 5400
large_middle
48229 22325 6666 6899
middle
21297 12104 5763
large_middle
45814 11277 7658 16894
unmapped
unmapped
unmapped
unmapped
right
10873 10086
right
25491 24797
unmapped
unmapped
right
1722 1628
left
1720 1470
unmapped
middle
17884 8739 5731
large_right
48253 7202 35848
unmapped
unmapped
unmapped
large_left
44569 32034 6290
unmapped
unmapped
large_middle
46453 12224 6618 17029
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
large_middle
43198 17651 7497 10780
large_left
44437 6683 32237
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
large_middle
33646 30737 0
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
large_middle
39167 2773 34057
unmapped
unmapped
left
11991 11855
unmapped
right
11421 10275
large_middle

...

unmapped
unmapped
large_middle
23355 6925 14088
unmapped
large_right
46201 6920 33705
unmapped
large_left
21129 6642 9466
left
7438 5239
unmapped
unmapped
large_right
44606 32777 6726
large_middle
45856 16640 7699 11518
unmapped
unmapped
large_right
48937 7122 36484
unmapped
large_middle
45842 17163 7589 11092
large_middle
30909 29338 0
unmapped
middle
6815 1975 2498
large_right
50736 30168 13385
large_middle
32016 4879 7211 14149
unmapped
unmapped
unmapped
unmapped
unmapped
large_middle
42662 36533 2685
unmapped
large_middle
29489 5460 7208 11048
left
4251 3069
unmapped
large_right
57181 28495 13191
unmapped
large_left
23777 6703 11847
unmapped
large_right
47137 34336 6785
middle
7473 5389 1719
large_middle
44190 7089 6687 22168
right
10467 7064
unmapped
large_middle
44452 12645 13276 9923
unmapped
large_middle
35652 8184 21852
unmapped
unmapped
unmapped
unmapped
unmapped
large_right
45997 7801 32531
right
3591 2248
unmapped
middle
16817 11344 2087
unmapped
unmapped
unmapped
unmapped


unmapped
large_middle
33963 18614 6739 2836
left
6561 6265
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
left
20671 20360
unmapped
left
7562 6523
unmapped
unmapped
unmapped
left
29353 24734
unmapped
large_left
21949 15969
middle
10594 2698 4497
unmapped
right
15711 13033
unmapped
unmapped
right
16407 12961
right
5007 4942
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
large_middle
33604 6917 24382
large_middle
21442 11096 7989
left
4263 3030
large_left
42699 7497 29168
unmapped
large_middle
27956 13728 10814
large_middle
47043 20719 6715 8417
unmapped
large_middle
23125 7134 13643
unmapped
large_middle
41773 1559 7294 26975
right
9052 6909
unmapped
large_middle
44500 7505 6690 21632
unmapped
large_middle
42781 20656 6687 8510
left
4286 3109
large_middle
41946 15962 6652 13286
unmapped
large_left
26346 7720 15761
unmapped
large_left
44190 7695 31796
unmapped
unmapped
unmapped
unmapped
unmappe

unmapped
middle
13681 4438 6897
left
11499 11022
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
large_middle
25319 5006 17955
large_middle
30148 995 26845
middle
13606 5638 4564
left
5813 3460
left
2700 1183
unmapped
unmapped
unmapped
middle
15463 8130 3950
large_left
39741 35835 2
unmapped
large_middle
22052 3664 16044
large_middle
41913 4705 6689 24461
unmapped
unmapped
unmapped
unmapped
unmapped
large_middle
40347 6775 6781 21031
unmapped
unmapped
large_middle
43454 20680 7566 7629
large_middle
34173 6370 10160 13718
unmapped
left
5060 4704
unmapped
unmapped
unmapped
large_middle
26549 7499 16724
unmapped
unmapped
unmapped
unmapped
right
9526 8167
unmapped
unmapped
unmapped
large_middle
27460 7891 15710 703
unmapped
unmapped
unmapped
unmapped
unmapped
large_left
26028 6738 15827
unmapped
unmapped
unmapped
unmapped
large_middle
22055 782 18927
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
right
4509 2808
large_middle
46632 17670 6664

unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
right
2831 2677
unmapped
unmapped
large_middle
24538 6989 15203
large_left
42041 4375 34051
unmapped
middle
13099 4572 7823
unmapped
unmapped
left
6037 3663
large_middle
41529 11733 7326 16710
unmapped
large_right
43408 29787 7494
unmapped
left
4120 1957
unmapped
unmapped
right
20265 19990
unmapped
unmapped
large_middle
41238 17428 6651 9645
large_middle
22319 12317 6589
right
18566 15838
unmapped
large_middle
21586 5367 8516 4416
unmapped
right
12003 9363
middle
10077 3373 3304
left
13601 12559
unmapped
unmapped
unmapped
unmapped
large_left
32298 6725 20636
large_middle
47871 24736 6732 4559
unmapped
unmapped
middle
4757 3065 1327
unmapped
unmapped
unmapped
large_left
44681 4466 35228
unmapped
unmapped
large_middle
24775 2750 19686
unmapped
unmapped
left
20773 17138
large_middle
22664 2155 16150 961
unmapped
right
3659 3252
right
5438 1953
middle
27014 2309 22369
left
9598 9369
unmapped
right
34799 33958
left
30221 28565
unmapped
u

unmapped
unmapped
middle
20282 2945 16626
large_middle
23874 4229 13917
unmapped
left
5924 1585
unmapped
unmapped
unmapped
unmapped
left
5931 5441
large_middle
35282 16881 7738 4888
unmapped
unmapped
large_middle
32974 20173 7195 3
unmapped
unmapped
large_middle
38785 32454 2912
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
large_middle
28271 7674 19881
unmapped
unmapped
unmapped
unmapped
unmapped
middle
21494 5604 13533
unmapped
unmapped
large_middle
34647 22097 7187 0
unmapped
right
2676 2207
large_middle
41445 3145 28434 4047
right
1245 1180
left
24958 22694
unmapped
unmapped
unmapped
large_left
20118 6718 9262
unmapped
unmapped
unmapped
unmapped
middle
20472 3590 14533
left
23912 23310
unmapped
unmapped
middle
5339 1892 3081
unmapped
unmapped
middle
18711 9260 6051
unmapped
unmapped
unmapped
unmapped
unmapped
unmapped
large_right
41332 1736 33753
left
28883 27679
unma

## Investigatvie section

In [25]:
SAM_FILE = pysam.AlignmentFile(sam_path, "r") # open samfile
# Note that this loops through alignments and not reads!
# Alignments don't seem to inlcude secondary alignment (read.is_secondary == True)
algn_dict = {}
for read in SAM_FILE:
    #print(read.query_name,read.query_length,read.query_alignment_start,read.query_alignment_end,read.query_alignment_length)
    # read.infer_query_length() includes reads that are unmapped as None for query length
    if read.query_name in algn_dict:
        algn_dict[read.query_name].append((read.query_alignment_start,read.query_alignment_end))
    else:
        algn_dict[read.query_name] = [read.infer_query_length(),(read.query_alignment_start,read.query_alignment_end)]

#print(algn_dict)

single_count = 0
unmapped_count = 0
left_count = 0
right_count = 0
middle_count = 0
two_middle_count = 0
two_both_count = 0
remainder_count = 0
large_count = 0
large_misalign_only = 0
large_single = 0
large_double = 0
large_more_than_two = 0
for i in algn_dict.keys():
    # Unmapped reads
    if algn_dict[i][0] == None:
        unmapped_count += 1
    # Single alignment
    elif len(algn_dict[i]) == 2:
        single_count += 1
        #print(algn_dict[i])
        if algn_dict[i][1][0] < 500:
            left_count += 1
        elif algn_dict[i][0]-algn_dict[i][1][1] < 500:
            right_count += 1
        elif algn_dict[i][1][1]-algn_dict[i][1][0] < 6500:
            middle_count += 1
    # Two alignments and >20 kb (mostly insert; changing to >10kb doesn't change numbers much)
#     elif len(algn_dict[i]) == 3 and algn_dict[i][0] > 20000:
#         # determine max and min values in alignment
#         #print(algn_dict[i])
#         max_left = max(algn_dict[i][1:],key=lambda item:item[1])[0]
#         min_left = min(algn_dict[i][1:],key=lambda item:item[1])[0]
#         min_right = min(algn_dict[i][1:],key=lambda item:item[1])[1]
#         max_right = max(algn_dict[i][1:],key=lambda item:item[1])[1]
#         # case where vector is at both ends and full insert is sequenced (98 of these)
#         if max_left-min_right > 20000:
#             #print(algn_dict[i])
#             #print("both",algn_dict[i][0],max_left,min_right)
#             two_both_count += 1
#         # case where vector is in middle
#         if max_right-min_left < 7000:
#             #print(algn_dict[i])            
#             #print("middle",algn_dict[i][0],min_left,max_right)
#             two_middle_count += 1
    elif algn_dict[i][0] > 20000:
        large_count += 1
        pos_start=[]
        pos_end=[]
        for pos in algn_dict[i][1:]:
            if pos[1]-pos[0] > 500:
                pos_start.append(pos[0])
                pos_end.append(pos[1])
        #print(pos_start)
        #print(pos_end)
        if len(pos_start) == 0:
            large_misalign_only += 1
            print(algn_dict[i])
            print()
            print("LARGE_MISALIGN_ONLY")
            print()
        if len(pos_start) == 1:
            large_single += 1
            if pos_start[0] < 500:
                print(algn_dict[i])
                print(pos_start)
                print(pos_end)
                print("large_left")
            elif algn_dict[i][0]-pos_end[0] < 500:
                print(algn_dict[i])
                print(pos_start)
                print(pos_end)
                print("large_right")
            elif pos_end[0]-pos_start[0] < 6500:
                print(algn_dict[i])
                print(pos_start)
                print(pos_end)
                print("large_middle")
        elif len(pos_start) == 2:
            large_double += 1
            if min(pos_start) < 1000:
                print(algn_dict[i])
                print(pos_start)
                print(pos_end)
                print("large_left")
            elif algn_dict[i][0]-max(pos_end) < 1000:
                print(algn_dict[i])
                print(pos_start)
                print(pos_end)
                print("large_right")
            else:
                print(algn_dict[i])
                print("0 to "+str(min(pos_start)))
                print(str(min(pos_end))+" to "+str(max(pos_start)))
                print(str(max(pos_end))+" to "+str(algn_dict[i][0]))
                print("large_middle")
        else:
            large_more_than_two += 1
    else:
        #print(algn_dict[i])
        remainder_count += 1
    # reads with >2 alignments seem to contain many misalignments?
#     if len(algn_dict[i]) > 2 and algn_dict[i][0] > 10000:
#         for h in algn_dict[i][1:]:
#             # one of alignments is close to start
#             if h[0] < 500:
#                 print(algn_dict[i])
#                 break
#             # one of alignments is close to end
#             elif algn_dict[i][0] - h[1] < 500:
#                 print(algn_dict[i])
#                 break
#         multiple_count += 1

    
#print(unmapped_count) 
#print(single_count)
#print(left_count)
#print(right_count)
#print(middle_count)
#print(both_count)
#print(two_middle_count)
#print(two_both_count)
print(large_misalign_only)
print(large_single)
print(large_double)
print(large_more_than_two)
print(large_count)
print(remainder_count)
print(len(algn_dict))
print(unmapped_count+single_count+two_middle_count+two_both_count+large_count+remainder_count)

# There are 137 reads with alignments above 5,000 bp and the longest is <5,732
# Most have split alignments longer than the vector sequence, some overlap, some have gaps
# Some align at both ends. There are:
#   - 5 reads with alignments 100 bp from either end -> all are vector only    
#   - 22 reads with alignments 250 bp from either end -> all are vector only
#   - 63 reads with alignments 500 bp from either end -> all are vector reads
# There are 12441 reads split as follows (using 500 bp as cutoff):
#   - 2375 reads with multiple alignments (and >20000 bp)
#      - commented out and folded into below (140 are > 20000 and have two alignments -> utilised)
#      - 1 is >20000 bp and has no (true) alignment >500 bp -> utilised
#      - 267 are >20000 bp and have one (true) alignment >500 bp -> utilised
#      - 1207 are >20000 bp and have two (true) alignments >500 bp -> utilised
#      - 901 are >20000 bp and have three or more (true) alignments >500 bp -> discarded
#   - 2024 reads with a single alignment (965 left, 623 right, 436 middle (all < 4000))
#   - 6433 unmapped reads
#   - 1609 remaining reads discarded (+901 from above to make total of 2510 (20.2%))

# Is this just a repeat of soft-clipping???
# Are there any homopolymers in the vector that may be confounding things due to seq errors?



[33974, (1287, 3648), (32807, 33879), (1168, 1291)]
[1287, 32807]
[3648, 33879]
large_right
[40559, (11923, 15322), (21957, 25980), (18596, 18946)]
0 to 11923
15322 to 21957
25980 to 40559
large_middle
[41316, (36877, 40300), (5686, 9775), (35624, 35992)]
0 to 5686
9775 to 36877
40300 to 41316
large_middle
[41974, (37738, 41149), (5856, 8217), (36118, 36482)]
[37738, 5856]
[41149, 8217]
large_right
[41395, (26430, 29865), (37030, 39389), (36529, 36916), (4860, 5216), (36914, 37033)]
0 to 26430
29865 to 37030
39389 to 41395
large_middle
[41203, (27126, 30548), (37254, 41203), (3943, 4303), (3, 134)]
[27126, 37254]
[30548, 41203]
large_right
[29252, (10174, 13616), (20809, 23167), (20314, 20695), (8932, 9298), (20693, 20812)]
0 to 10174
13616 to 20809
23167 to 29252
large_middle
[28321, (9167, 12571), (19861, 22205), (19373, 19755), (8972, 9332), (19753, 19865)]
0 to 9167
12571 to 19861
22205 to 28321
large_middle
[40009, (22284, 25698), (32412, 34768), (7597, 7961)]
0 to 22284
25698 to 

37645 to 40240
large_middle
[45491, (14464, 20147), (39167, 39874), (31041, 31410), (32493, 32855)]
0 to 14464
20147 to 39167
39874 to 45491
large_middle
[34233, (692, 3122), (33691, 34233), (539, 663)]
[692, 33691]
[3122, 34233]
large_left
[40866, (37248, 40624), (337, 2648), (0, 238), (40647, 40866), (216, 341)]
[37248, 337]
[40624, 2648]
large_left
[22924, (7069, 7767), (271, 619)]
[7069]
[7767]
large_middle
[28916, (18188, 21608), (28212, 28599), (698, 1060), (28711, 28914), (28601, 28714)]
[18188]
[21608]
large_middle
[31068, (20374, 23795), (5533, 6463), (4070, 4452), (6947, 7273)]
0 to 5533
6463 to 20374
23795 to 31068
large_middle
[48267, (38891, 42287), (722, 3074), (42301, 42667)]
[38891, 722]
[42287, 3074]
large_left
[45243, (41467, 44988), (6537, 8891), (38706, 39058)]
[41467, 6537]
[44988, 8891]
large_right
[49071, (14193, 17574), (24176, 26508), (17576, 17934)]
0 to 14193
17574 to 24176
26508 to 49071
large_middle
[39586, (14747, 18179), (24913, 27276), (18175, 18535)]
0 

[43858, (32035, 35437), (42060, 43858), (4, 389), (1818, 2135)]
[32035, 42060]
[35437, 43858]
large_right
[41986, (16271, 19699), (26422, 28765), (15564, 15928)]
0 to 16271
19699 to 26422
28765 to 41986
large_middle
[41804, (19920, 23326), (30685, 33047), (30186, 30569), (11642, 12006), (30567, 30689)]
0 to 19920
23326 to 30685
33047 to 41804
large_middle
[45835, (6139, 8488), (44123, 45835)]
[6139, 44123]
[8488, 45835]
large_right
[26240, (11436, 14871), (21630, 23981), (14868, 15234)]
0 to 11436
14871 to 21630
23981 to 26240
large_middle
[45947, (30416, 33842), (40563, 42910), (33838, 34197)]
0 to 30416
33842 to 40563
42910 to 45947
large_middle
[47426, (8396, 11727), (18282, 20576), (11735, 12099)]
0 to 8396
11727 to 18282
20576 to 47426
large_middle
[41172, (24699, 28110), (34635, 36924), (6537, 6876)]
0 to 24699
28110 to 34635
36924 to 41172
large_middle
[35074, (11581, 13934), (30187, 30550), (23493, 23853)]
[11581]
[13934]
large_middle
[34708, (27367, 30800), (2161, 3094), (704,

[5401]
large_middle
[43551, (35292, 38705), (2706, 5061)]
0 to 2706
5061 to 35292
38705 to 43551
large_middle
[29002, (3574, 6983), (13695, 16043), (6991, 7356)]
0 to 3574
6983 to 13695
16043 to 29002
large_middle
[22817, (155, 2508), (22662, 22817)]
[155]
[2508]
large_left
[31200, (0, 2219), (9581, 11942), (9079, 9466), (22145, 22516), (9464, 9585)]
[0, 9581]
[2219, 11942]
large_left
[38651, (23371, 26761), (2016, 2946), (26791, 27161), (33484, 33858), (3419, 3737)]
0 to 2016
2946 to 23371
26761 to 38651
large_middle
[30199, (393, 2733), (0, 385)]
[393]
[2733]
large_left
[24223, (12894, 16325), (23026, 23957), (1191, 1553)]
[12894, 23026]
[16325, 23957]
large_right
[25974, (1731, 4076), (24243, 24597)]
[1731]
[4076]
large_middle
[43547, (37459, 40878), (4843, 7121)]
0 to 4843
7121 to 37459
40878 to 43547
large_middle
[44311, (35195, 38556), (1229, 3471), (43204, 43541), (1104, 1227)]
0 to 1229
3471 to 35195
38556 to 44311
large_middle
[49027, (34420, 37854), (44581, 46927), (4454, 481

large_middle
[44154, (19238, 22632), (29293, 31605), (14861, 15177)]
0 to 19238
22632 to 29293
31605 to 44154
large_middle
[22531, (654, 2999), (21874, 22255)]
[654]
[2999]
large_middle
[47945, (18424, 21795), (28471, 30808), (21803, 22162)]
0 to 18424
21795 to 28471
30808 to 47945
large_middle
[30952, (1892, 4211), (29060, 29405)]
[1892]
[4211]
large_middle
[30195, (9678, 13103), (19806, 23903), (10383, 10748)]
0 to 9678
13103 to 19806
23903 to 30195
large_middle
[44036, (35558, 38980), (1598, 3915), (42438, 42794)]
0 to 1598
3915 to 35558
38980 to 44036
large_middle
[22577, (8107, 10449), (0, 1384), (1380, 1741)]
[8107, 0]
[10449, 1384]
large_left
[42917, (29197, 34861), (21831, 22534), (15065, 15429), (13719, 13830), (21083, 21195)]
0 to 21831
22534 to 29197
34861 to 42917
large_middle
[37834, (15834, 19254), (26808, 29160)]
0 to 15834
19254 to 26808
29160 to 37834
large_middle
[45438, (15462, 18840), (25474, 27781), (18836, 19194)]
0 to 15462
18840 to 25474
27781 to 45438
large_mid

[17037]
large_middle
[34697, (677, 4111), (10870, 13228), (4107, 4471)]
[677, 10870]
[4111, 13228]
large_left
[25681, (1433, 3784), (939, 1315), (24736, 25100), (1316, 1436)]
[1433]
[3784]
large_middle
[41689, (14338, 17706), (24299, 26636), (17390, 17749)]
0 to 14338
17706 to 24299
26636 to 41689
large_middle
[46324, (4335, 7762), (14474, 16817), (7758, 8125)]
0 to 4335
7762 to 14474
16817 to 46324
large_middle
[21255, (2548, 4886), (18706, 19059)]
[2548]
[4886]
large_middle
[45656, (17779, 21209), (27897, 30204), (21205, 21570)]
0 to 17779
21209 to 27897
30204 to 45656
large_middle
[22544, (2585, 4938), (19959, 20317)]
[2585]
[4938]
large_middle
[20509, (7952, 13654), (13999, 14358), (12556, 12919)]
[7952]
[13654]
large_middle
[45645, (18319, 21735), (29365, 31707), (16277, 16661), (23415, 23798), (22224, 22579), (23796, 23909)]
0 to 18319
21735 to 29365
31707 to 45645
large_middle
[23323, (1516, 4942), (12155, 14485), (11646, 12026), (11671, 12039), (12024, 12135)]
0 to 1516
4942 to

0 to 16735
20181 to 27744
30105 to 43601
large_middle
[43281, (22653, 26054), (33526, 35840)]
0 to 22653
26054 to 33526
35840 to 43281
large_middle
[28871, (24992, 28428), (118, 443)]
[24992]
[28428]
large_right
[44558, (31190, 34611), (41382, 43720), (3190, 3591)]
[31190, 41382]
[34611, 43720]
large_right
[27332, (13776, 17187), (23893, 26251), (17183, 17547)]
0 to 13776
17187 to 23893
26251 to 27332
large_middle
[43902, (26326, 29735), (6724, 7226), (6340, 6726), (7339, 7695), (7224, 7345)]
0 to 6724
7226 to 26326
29735 to 43902
large_middle
[23751, (12665, 15026), (1415, 2345), (3141, 3498), (2825, 3149), (0, 323)]
0 to 1415
2345 to 12665
15026 to 23751
large_middle
[23559, (2576, 4922), (21104, 21817), (2452, 2580)]
0 to 2576
4922 to 21104
21817 to 23559
large_middle
[38489, (689, 2991), (37950, 38489), (534, 660)]
[689, 37950]
[2991, 38489]
large_left
[43639, (17170, 20581), (28155, 30506)]
0 to 17170
20581 to 28155
30506 to 43639
large_middle
[23874, (4229, 9957), (19644, 20023),

In [17]:
max(pos_start)

35032